# Example side view and focus metric trace

In [ ]:
from msr_reader import OBFFile
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter1d
from skimage.exposure import rescale_intensity
from skimage.io import imsave

from calmutils.color import gray_images_to_rgb_composite
from calmutils.misc.visualization import get_orthogonal_projections_8bit

in_file = 'compaction_sted_data/39_KB_20171213/A92_KB_K562_wt_Dead11B-Atto594_Dead11A-Atto647N/K562_wt/raw/dd78e085753290968d623de3ee7dc8a6_field4.msr'

with OBFFile(in_file) as reader:
    imgs = [reader.read_stack(i) for i in (0, 1)]
    pixel_size = reader.pixel_size(0)

# get normalized, smoothed mean along z
trace = np.mean(imgs, axis=(0,2,3))
trace = gaussian_filter1d(trace, 3, mode='constant')
trace /= trace.max()

projs = [get_orthogonal_projections_8bit(img, pixel_size=pixel_size) for img in imgs]
rgb_projections = gray_images_to_rgb_composite(projs, color_names=['cyan', 'magenta'])


fig, ax = plt.subplots(figsize = (5,5))
ax.imshow(rgb_projections)

imsave('autofocus_proj.png', (rgb_projections * 255).astype(np.uint8))

with plt.rc_context({'pdf.fonttype': 42}):
    fig,ax = plt.subplots(figsize=(6,2))
    ax.plot(trace)
    plt.savefig('autofocus_trace.pdf')


In [ ]:
from h5py import File
import json
from calmutils.misc.json import recursive_dict_query

h5_path = 'compaction_sted_data/63_KB_20190319/A209_K562-wt_Dead11B-594_Dead11C-647N/K562_wt_3d/raw/f2dba62714c8d1ccf11e23410c7f0b6b.h5'
image_id = 'field8'

with File(h5_path) as h5_file:

    # load channel 0, 1 imgs
    imgs = [h5_file[f'experiment/{image_id}/0/{i}'][...].squeeze() for i in (0, 1)]

    # get pixel size from metadata
    attrs = json.loads(h5_file[f'experiment/{image_id}/0'].attrs['measurement_meta'])
    pixel_size = [float(recursive_dict_query(attrs, f'ExpControl/scan/range/{d}/psz')) for d in 'zyx']


# get normalized, smoothed mean along z
trace = np.mean(imgs, axis=(0,2,3))
trace = gaussian_filter1d(trace, 3, mode='constant')
trace /= trace.max()

imgs = [rescale_intensity(img.astype(float), in_range=(0, 8)) for img in imgs]
yprojs = [img.max(axis=1) for img in imgs]

plt.imshow(gray_images_to_rgb_composite(yprojs, color_names=['cyan', 'magenta']), aspect=pixel_size[0]/pixel_size[1])

plt.figure()
plt.plot(trace)

# Focus position map

Here, we produce a scatterplot/heatmap of focus positions of an acquisition.

First, we find the acquisition with the most overviews/fields:

In [ ]:
import pandas as pd

data_path = '/Users/david/Desktop/scratch_data/combined_analysis_3d.csv'
df = pd.read_csv(data_path)

# create table of run_idx, field_idx, detail_idx from filenames
image_count_df = df.file.str.split('/', expand=True)[3].str.extract('(.*?)_field_([0-9]+)_sted_([0-9]+).msr')
image_count_df.columns = ['run_idx', 'field_idx', 'detail_idx']

# treat field, detail idxs as numbers
image_count_df['field_idx'] = image_count_df.field_idx.astype(int)
image_count_df['detail_idx'] = image_count_df.detail_idx.astype(int)

# add slide (group) etc. from original df
image_count_df = pd.concat([df[['slide_id', 'slide_group_id', 'acquisition_id']], image_count_df], axis=1)

# get maximum field idx for each acquisition, sort
image_count_df.groupby(['slide_id', 'slide_group_id', 'acquisition_id']).field_idx.max().sort_values()

## 1.2) Load and plot z-position from largest replicate

In [ ]:
from calmutils.misc.json import recursive_dict_query
from h5py import File
import json
from collections import defaultdict
import seaborn as sns
from matplotlib import pyplot as plt


h5_path = 'compaction_sted_data//63_KB_20190319/A209_K562-wt_Dead11B-594_Dead11C-647N/K562_wt_3d/raw/f2dba62714c8d1ccf11e23410c7f0b6b.h5'

df_pos = defaultdict(list)
with File(h5_path) as h5_file:
    for ov_key in filter(lambda s: 'sted' not in s, h5_file['experiment'].keys()):
        attrs = json.loads(h5_file[f'experiment/{ov_key}/0'].attrs['measurement_meta'])

        z_stage = float(recursive_dict_query(attrs, 'ExpControl/scan/range/offsets/coarse/z/g_off'))
        y_stage = float(recursive_dict_query(attrs, 'ExpControl/scan/range/offsets/coarse/y/g_off'))
        x_stage = float(recursive_dict_query(attrs, 'ExpControl/scan/range/offsets/coarse/x/g_off'))

        df_pos['key'].append(ov_key)
        df_pos['z_pos_um'].append(z_stage * 1e6)
        df_pos['y_pos_mm'].append(y_stage * 1e3)
        df_pos['x_pos_mm'].append(x_stage * 1e3)

df_pos = pd.DataFrame(df_pos)
get_id = lambda s: int(''.join(c for c in s if c.isnumeric()))
df_pos['field_idx'] = df_pos.key.apply(get_id)

df_pos = df_pos.sort_values('field_idx').reset_index(drop=True)

# make positions relative to first field
df_pos[['z_pos_um', 'y_pos_mm', 'x_pos_mm']] -= df_pos[['z_pos_um', 'y_pos_mm', 'x_pos_mm']].loc[0]


with plt.rc_context({'pdf.fonttype': 42}):
    fig, ax = plt.subplots(figsize=(5,5))
    sns.scatterplot(df_pos, x='x_pos_mm', y='y_pos_mm', hue='z_pos_um', s = 90, ax=ax, palette=sns.palettes.color_palette('coolwarm', as_cmap=True))
    ax.set_xlim((-0.9, 0.9))
    ax.set_ylim((-0.9, 0.9))

    plt.savefig('autofocus_grid.pdf')

